In [1]:
from neuralprophet import NeuralProphet
from neuralprophet.utils import set_random_seed

KeyboardInterrupt: 

In [1]:
import pandas as pd 
from sqlite3 import connect
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from warnings import simplefilter
from scipy.signal import detrend
import matplotlib as mpl
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
sns.set()
import pandasql as psql
#machine learning and statistical methods
import statsmodels.api as sm
import math
#dataframe index manipulations
import datetime
from datetime import datetime
#selected preprocessing and evaluation methods
from sklearn.preprocessing import StandardScaler
from statsmodels.tsa.stattools import kpss
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from plotly.subplots import make_subplots
from statsmodels.tsa.seasonal import DecomposeResult
import plotly.graph_objects as go
from statsmodels.tsa.holtwinters import SimpleExpSmoothing, ExponentialSmoothing
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.metrics import mean_absolute_percentage_error
from prophet import Prophet
from neuralprophet import NeuralProphet
import tensorflow as tf
import torch 
from sklearn.model_selection import TimeSeriesSplit

2023-08-14 16:31:32.467777: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [ ]:
formtemplatesclones=pd.read_csv("/home/batuhan-saylam/Desktop/JotformProject/formTemplatesClones.csv",parse_dates=["date"])
formtemplatescategories=pd.read_csv("/home/batuhan-saylam/Desktop/JotformProject/formTemplatesCategories.csv")
formtemplates=pd.read_excel("/home/batuhan-saylam/Desktop/JotformProject/formTemplates_1_.xlsx")

In [ ]:




formtemplatescategoriesPharmacy=formtemplatescategories[formtemplatescategories["metaKeywords"]=='Tax Forms']

formtemplatesclones=formtemplatesclones.rename({"templateID":"_id"},axis=1)
formtemplates=formtemplates.drop_duplicates()
formandclone=pd.merge(formtemplates,formtemplatesclones,on="_id")
formtemplates=formtemplates.rename({"_id":"id"},axis=1)
formtemplates=formtemplates.rename({"_featuredCategory":"_id"},axis=1)
formtemplates2=pd.merge(formtemplatescategoriesPharmacy,formtemplates,on="_id")
formtemplates2=formtemplates2.rename({"_id":"_featuredCategory"},axis=1)
formtemplates=formtemplates.rename({"_id":"_featuredCategory"},axis=1)
df_melt = formtemplates.assign(_categories=formtemplates._categories.str.split(","))
formtemplates=df_melt._categories.apply(pd.Series) \
    .merge(formtemplates, right_index=True, left_index=True) \
    .drop(["_categories"], axis=1) \
    .melt(id_vars=['id',"_title","_slug","_description","_featuredCategory","_language"], value_name="_categories") \
    .drop("variable", axis=1) 
formtemplates=formtemplates.rename({"_categories":"_id"},axis=1)
formtemplates3=pd.merge(formtemplatescategoriesPharmacy,formtemplates,on="_id")
formtemplates3=formtemplates3.rename({"_id":"_categories"},axis=1)
formtemplates=formtemplates.rename({"_id":"_categories"},axis=1)
formtemplates=formtemplates.rename({"id":"_id"},axis=1)
formcat=pd.concat([formtemplates2,formtemplates3])
formcat=formcat.rename({"id":"_id"},axis=1)
formcat=formcat.loc[:,[
 'name',
 'description',
 'metaKeywords',
 '_categories',
 '_id']]
formandclone=formandclone.loc[:,[
'_id',
 '_title',
 '_slug',
 '_description',
 '_language',
 '_featuredCategory',
 'formID',
 'form_type',
 'source',
 'date']]
formclonecategnonnanPharmacy=pd.merge(formandclone,formcat,on="_id")
formclonecategnonnanPharmacy=formclonecategnonnanPharmacy.drop_duplicates(keep='first')

formclonecategnonnanPharmacy.index=formclonecategnonnanPharmacy.date
formclonecategnonnanPharmacy.index=formclonecategnonnanPharmacy.index.to_period("D")
formclonecategnonnanPharmacy=formclonecategnonnanPharmacy.rename({"date":"date2"},axis=1)
formclonecategnonnanPharmacy["day"]=formclonecategnonnanPharmacy.index.dayofweek
formclonecategnonnanPharmacy["week"]=formclonecategnonnanPharmacy.index.week
formclonecategnonnanPharmacy["dayofyear"]=formclonecategnonnanPharmacy.index.dayofyear
formclonecategnonnanPharmacy["year"]=formclonecategnonnanPharmacy.index.year
formclonecategnonnanPharmacy["quarter"]=formclonecategnonnanPharmacy.index.quarter
formclonecategnonnanPharmacy["month"]=formclonecategnonnanPharmacy.index.month
formclonecategnonnanPharmacy.index=formclonecategnonnanPharmacy.index.to_timestamp()






In [ ]:
from pytorch_lightning import Trainer

In [2]:
from lightning.pytorch.accelerators import find_usable_cuda_devices

# Find two GPUs on the system that are not already occupied
trainer = Trainer(accelerator="cuda", devices=find_usable_cuda_devices(1))

NameError: name 'Trainer' is not defined

In [3]:
use_cuda = torch.cuda.is_available()
if use_cuda:
    print('__CUDNN VERSION:', torch.backends.cudnn.version())
    print('__Number CUDA Devices:', torch.cuda.device_count())
    print('__CUDA Device Name:',torch.cuda.get_device_name(0))
    print('__CUDA Device Total Memory [GB]:',torch.cuda.get_device_properties(0).total_memory/1e9)
use_cuda

__CUDNN VERSION: 8700
__Number CUDA Devices: 1
__CUDA Device Name: NVIDIA GeForce RTX 3050 Laptop GPU
__CUDA Device Total Memory [GB]: 4.094164992


True

In [ ]:
df=psql.sqldf("select count(_id),date,year,dayofyear from formclonecategnonnanPharmacy  group by date")

df.index=df.date.tolist()
df.index=pd.to_datetime(df.index)
df.index=df.index.to_period("D")
df.index=df.index.to_timestamp()
df.index=df.index.strftime('%d-%m-%Y')
df["date"]=pd.to_datetime(df["date"])


df=df.rename({"count(_id)":"y"},axis=1)
df=df.rename({"date":"ds"},axis=1)
df=df.loc[:,["y","ds"]]





In [ ]:
tss = TimeSeriesSplit(n_splits = 5)
days = np.sort(df.index.unique())
for train_index, test_index in tss.split(df):
    train_days, test_days = days[train_index], days[test_index]
    X_train, X_test = df.loc[train_days,], df.loc[test_days,]

In [ ]:
from pytorch_lightning.callbacks import EarlyStopping
# Init ModelCheckpoint callback, monitoring 'val_loss'
checkpoint_callback = EarlyStopping(monitor="Loss",mode="min",patience=10)



In [ ]:
from pytorch_lightning.callbacks import ModelCheckpoint

In [ ]:
import pytorch_lightning as pl
from lightning.pytorch.callbacks.early_stopping import EarlyStopping

class _EarlyStopping(EarlyStopping, pl.Callback):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

early_stop_callback = _EarlyStopping(monitor="Loss",  mode="min")

In [ ]:
from lightning.pytorch.callbacks import EarlyStopping
checkpoint_callback2 = EarlyStopping(monitor="Loss",mode="min")

In [ ]:
set_random_seed(0)
m = NeuralProphet(trainer_config={"accelerator":"gpu","check_val_every_n_epoch":1,"callbacks":[checkpoint_callback]},daily_seasonality=False,epochs=1000)
m.add_country_holidays(country_name='US')
df_model=m.fit(X_train, freq="D",early_stopping=True,checkpointing=True,learning_rate=,batch_size=8)

WARNING - (NP.forecaster.fit) - When Global modeling with local normalization, metrics are displayed in normalized scale.
INFO - (NP.df_utils._infer_frequency) - Major frequency YS corresponds to 71.337% of the data.
WARNING - (NP.df_utils._infer_frequency) - Defined frequency D is different than major frequency YS
INFO - (NP.config.init_data_params) - Setting normalization to global as only one dataframe provided for training.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3050 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
Loading `train_dataloader` to estimate nu

Training: 0it [00:00, ?it/s]

In [ ]:
path="/home/batuhan-saylam/Desktop/lightning_logs/version_2/checkpoints/epoch=516-step=15510.ckpt"

In [ ]:
torch.load(path)

{'epoch': 516,
 'global_step': 15510,
 'pytorch-lightning_version': '1.9.5',
 'state_dict': OrderedDict([('trend.bias', tensor([-0.2320], device='cuda:0')),
              ('trend.trend_k0', tensor([[[0.7220]]], device='cuda:0')),
              ('trend.trend_deltas',
               tensor([[[ 1.0398,  0.4710, -0.3245, -1.2271,  1.0922,  0.4679, -0.5985,
                          0.1880,  1.5828,  1.7215, -0.7956]]], device='cuda:0')),
              ('trend.trend_changepoints_t',
               tensor([0.0000, 0.0727, 0.1455, 0.2182, 0.2909, 0.3636, 0.4364, 0.5091, 0.5818,
                       0.6545, 0.7273], device='cuda:0')),
              ('seasonality.season_params.weekly',
               tensor([[[-0.0972,  0.0429,  0.0467,  0.0049, -0.0113, -0.0013]]],
                      device='cuda:0')),
              ('seasonality.season_params.yearly',
               tensor([[[0.0737, 0.2516, 0.0950, 0.0817, 0.0566, 0.0636, 0.0497, 0.0262,
                         0.0578, 0.0246, 0.0212, 